# Build the Neural Network

Neural networks consists of interconnected layers or components that process information. The `torch.nn` namespace supplies all the essential elements for creating a custom neural network. In PyTorch, every module is a subclass of nn.Module. A neural network is itself a module made up of other modules(layers), allowing for the simple construction and organization of intricate architectures.

In the subsequenct segments, we will develop a neural network for categorizing images within the FashionMNIST dataset.

In [1]:
import os
import torch
# Importing the necessary building blocks to create and work with neural network
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [3]:
# DEFINE THE CLASS

# We define our neural network by subclassing `nn.Module`, and initialize the neural network layers in __init__. Every `nn.Module` subclass implements the operations on input data in the `forward` method
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [4]:
# We create an instance of `NeuralNetwork`, and move it to the `device`, and print its structure
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
# To use the model, we pass it the input data. This executes the model's `forward`, along with some background operations.
# Do not call `model.forward()` directly.

# Calling the model on the input returns a 2-dimensional tensor with dim = 0 corresponding to each output of 10 raw predicted values for each class, and dim = 1 corresponding to the individual values of each output. We get the prediction probabilities by passing it through an instance of the `nn.Softmax` module.
X = torch.rand(1, 28, 28, device = device)
logits = model(X)
pred_probab = nn.Softmax(dim = 1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7], device='cuda:0')


In [6]:
# MODEL LAYERS

# Let's break down the layrs in the FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [7]:
# nn.Flatten

# We initialize the `nn.Flatten` layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (the minibatch dimension (at dim = 0) is maintained)
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [8]:
# nn.Linear

# The linear layer is a module that applies a linear transformation on the input usin its stored weights and biases
layer1 = nn.Linear(in_features = 28*28, out_features = 20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [9]:
# nn.ReLU

# Non-linear activation are what create the complex mappings between the model's inputs and outputs. They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena

# In this model, use nn.ReLU between our linear layers, but there's other activations to introduce non-linearity in your model
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.3235, -0.1688,  0.6223, -0.2024,  0.0145,  0.0499,  0.0394,  0.2611,
          0.0346, -0.1341, -0.7167, -0.0926,  0.1514, -0.3075, -0.0142,  0.2741,
          0.0010,  0.2832, -0.4210, -0.1214],
        [ 0.5129,  0.1634,  0.4923, -0.2584,  0.0060,  0.0848, -0.1998,  0.0027,
          0.1571, -0.1094, -0.8463, -0.2198, -0.1086, -0.4119,  0.2210,  0.4198,
         -0.0268,  0.6057, -0.1111, -0.2832],
        [ 0.1763,  0.1407,  0.6262, -0.0722,  0.2622, -0.2901, -0.1397,  0.1571,
         -0.1506, -0.4649, -0.8416, -0.1729,  0.2107, -0.2799,  0.5045,  0.1497,
          0.0786,  0.5266, -0.3103,  0.1906]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.3235, 0.0000, 0.6223, 0.0000, 0.0145, 0.0499, 0.0394, 0.2611, 0.0346,
         0.0000, 0.0000, 0.0000, 0.1514, 0.0000, 0.0000, 0.2741, 0.0010, 0.2832,
         0.0000, 0.0000],
        [0.5129, 0.1634, 0.4923, 0.0000, 0.0060, 0.0848, 0.0000, 0.0027, 0.1571,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.22

In [10]:
# nn.Sequential

# `nn.Sequential` is an ordered container of modules. The data is passedthrough all the modules in the same order as defined. You can use sequential containers to put together a quick network like `seq_modules`
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10),
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [12]:
# nn.Softmax

#The last linear layer of the neural network returns logits raw values in [-inf, inf] which are passed to the nn.Softmax module.
# The logits are scaled to values [0,1] representing the model's predicted probabilities for each class.
# `dim` parameter indicates the dimension along which the values must sum to 1.
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [13]:
# MODEL PARAMETERS

# Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized durin training.
# Subclassing nn.Module automatically tracks all fields defined inside your model bject, and makes all parameters accessible using your model's `parameters()` or `named_parameters()` methods.
# In this example, we iterate over each parameter, and print its size and a preview of its values
print(f"Model structure: {model}\n\n")
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]}\n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0215,  0.0169, -0.0085,  ...,  0.0133,  0.0209, -0.0032],
        [-0.0245,  0.0083, -0.0341,  ...,  0.0233,  0.0300,  0.0186]],
       device='cuda:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0242,  0.0079], device='cuda:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0038, -0.0028, -0.0295,  ..., -0.0390,  0.0376,  0.0193],
        [ 0.0395, -0.0095,  0.0047,  ...,  0.0318, -0.0205,  0.0034]],
       device='cuda:0', grad_fn=<Slic